In [ ]:
order=10

In [ ]:
# import libraries
import glob                               #can be used to find the target files
import matplotlib.pyplot as plt
import nibabel as nib                   # to read and write neuron imaging data 
import numpy as np                        # for matrix calculating
import subprocess
from PIL import Image 
import os,shutil

from scipy import signal
from skimage import color, img_as_ubyte, io

In [ ]:
file_script = 'results_script1'
if not os.path.exists(file_script):
    os.mkdir(file_script)

In [ ]:
shutil.copy('/home/zhiwei/Desktop/semester-projects/original_file/3000/BVECTOR.scheme','results_script1')
shutil.copy('/home/zhiwei/Desktop/semester-projects/original_file/3000/fsltensor_FA.nii.gz','results_script1')
shutil.copy('/home/zhiwei/Desktop/semester-projects/original_file/3000/fsltensor_V3.nii.gz','results_script1')

CMD = 'shutil.copy(“fsltensor_V1.nii.gz”,”results_script1”)'
subprocess.call(CMD,shell=True)


In [ ]:
cd results_script1

## generate original tractography and get fib_original, hdr_original

In [ ]:
data = nib.load('fsltensor_FA.nii.gz')
d=np.ones(data.get_data().shape)
img=nib.Nifti1Image(d, data.affine)
nib.save(img,'mask.nii')

In [ ]:
CMD='tckgen fsltensor_V3.nii.gz origin0.015.tck -algorithm FACT -seed_image mask.nii -mask mask.nii -select 500 -step 0.015'
subprocess.call(CMD, shell = True)

In [ ]:
CMD = 'TractConverter.py -i origin0.015.tck -o origin0.015.trk -a fsltensor_FA.nii.gz'
subprocess.call(CMD,shell=True)

In [ ]:
fib_original,hdr_original=nib.trackvis.read('/home/zhiwei/Desktop/semester-projects/results_script1/origin0.015.trk')

## generate synthetic CLARITY data

In [ ]:
CMD='tckmap origin0.015.tck syn_CLARITY.nii.gz -vox 0.015'
subprocess.call(CMD, shell = True)


## calculate structure tensor

In [ ]:
def doggen(sigma=None, X=None, Y=None, Z=None):
    halfsize = np.ceil(3 * np.max(sigma))       #3*the maximum number of sequence sigma, ceil: shangxian
    x = np.arange(-halfsize,halfsize+1)         # sequence from -halfsize to halfsize+1

    if len(sigma) == 1:
        if X is None:
            X = x
        k = (-1)*X * np.exp( (-1)*np.power(X,2)/(2 * np.power(sigma[0],2)) )
    if len(sigma) == 2:
        if X is None or Y is None:
            [X,Y] =np.meshgrid(x,x)
        k = (-1)*X * np.exp( (-1)*np.power(X,2)/(2 * np.power(sigma[0],2)) ) * np.exp( (-1)*np.power(Y,2)/(2 * np.power(sigma[1],2)) )
    if len(sigma) == 3:
        if X is None or Y is None or Z is None:
            [X,Y,Z] =np.meshgrid(x,x,x)
        k = (-1)*X * np.exp( (-1)*np.power(X,2)/(2 * np.power(sigma[0],2)) ) * np.exp( (-1)*np.power(Y,2)/(2 * np.power(sigma[1],2)) ) * np.exp( (-1)*np.power(Z,2)/(2 * np.power(sigma[2],2)))
    if len(sigma) > 3:
        print ('Only support up to dimension 3')
    
    return k /np.sum(np.abs(k))

def gradCompute(img, dogsigma):
    dogkercc = doggen([dogsigma, dogsigma, dogsigma])
    dogkerrr = np.transpose( dogkercc, (1, 0, 2) )
    dogkerzz = np.transpose( dogkercc, (0, 2, 1) )

    gcc = signal.convolve(img, dogkercc, mode = 'valid')
    grr = signal.convolve(img, dogkerrr, mode = 'valid')
    gzz = signal.convolve(img, dogkerzz, mode = 'valid')

    # Gradient products
    gp = type('', (), {})()
    gp.gprrrr = grr * grr
    gp.gprrcc = grr * gcc
    gp.gprrzz = grr * gzz
    gp.gpcccc = gcc * gcc
    gp.gpcczz = gcc * gzz
    gp.gpzzzz = gzz * gzz

    # Gradient amplitude
    ga = np.sqrt(gp.gprrrr + gp.gpcccc + gp.gpzzzz)

    return ga, gp



## the optimal patch_size to retrieve the exact size of CLARITY as the originally synthetic one!! do not change this anymore!!

In [ ]:
patch_size = 6.6

In [ ]:

img_stack = nib.load('syn_CLARITY.nii.gz')
imgstack = img_stack.get_fdata()/255.0


# select sigma
dogsigma = [0.6]
vox_size = [15, 15, 15] #um
#define the voxel dimension to apply tensor
voxel_dim = np.array([patch_size,patch_size,patch_size]) # x,y,z in voxel dimension
volume_dim = (np.array(imgstack.shape)/voxel_dim).astype(np.int) # in voxel dimesion: removed voxel at the border

fsl_tensor = np.zeros((volume_dim[0], volume_dim[1], volume_dim[2], 6))
for x in range (0,volume_dim[0]):
    for y in range (0,volume_dim[1]):
        for z in range (0, volume_dim[2]):
            [ga,gp] = gradCompute(imgstack[int(x*voxel_dim[0]):int((x+1)*voxel_dim[0]), int(y*voxel_dim[1]):int((y+1)*voxel_dim[1]), int(z*voxel_dim[2]):int((z+1)*voxel_dim[2])], dogsigma)

            lowthreh = np.percentile(ga.reshape(-1), 20)
            highthreh = np.percentile(ga.reshape(-1), 80)

            mask = np.ones(ga.shape)
            mask[ga  < lowthreh] = 0
            mask[ga  > highthreh] = 0

            gprrrr = np.sum(gp.gprrrr[mask==1])
            gprrcc = np.sum(gp.gprrcc[mask==1])
            gprrzz = np.sum(gp.gprrzz[mask==1])
            gpcccc = np.sum(gp.gpcccc[mask==1])
            gpcczz = np.sum(gp.gpcczz[mask==1])
            gpzzzz = np.sum(gp.gpzzzz[mask==1])

            fsl_tensor[x,y,z] = np.array([[gprrrr, gprrcc, gprrzz,gpcccc, gpcczz, gpzzzz]])
affine = np.eye(4)
affine[0,0]=vox_size[0]*voxel_dim[0]*0.001
affine[1,1]=vox_size[1]*voxel_dim[1]*0.001
affine[2,2]=vox_size[2]*voxel_dim[2]*0.001
img_fsl_tensor = nib.Nifti1Image(fsl_tensor, affine)
nib.save(img_fsl_tensor, 'fsltensor_ftrial.nii.gz')
CMD='fsl5.0-fslmaths fsltensor_ftrial.nii.gz -tensor_decomp fsltensor_ftrial'
subprocess.call(CMD, shell=True)

## generate the mask for subsequent tractography generation

In [ ]:
data = nib.load('fsltensor_ftrial_FA.nii.gz')
d=np.ones(data.get_data().shape)
img=nib.Nifti1Image(d, data.affine)
nib.save(img,'mask_syn.nii')

##  generate the tractography and subsequent CLARITY based on the synthetic CLARITY images

In [ ]:
number = 100000

In [ ]:
CMD='tckgen fsltensor_ftrial_V3.nii.gz before0.015.tck -algorithm FACT -seed_image mask_syn.nii -mask mask_syn.nii -select 100000 -step 0.015'
subprocess.call(CMD, shell = True)

In [ ]:
CMD = 'TractConverter.py -i before0.015.tck -o before0.015.trk -a fsltensor_ftrial_FA.nii.gz'
subprocess.call(CMD,shell = True)

In [ ]:
fib_before,hdr_before=nib.trackvis.read('/home/zhiwei/Desktop/semester-projects/results_script1/before0.015.trk')

In [ ]:
for items in range(len(fib_original)):
    fib_before.append(fib_original[items])
nib.trackvis.write('fibers_before_all.trk',fib_before,hdr_before)

In [ ]:
CMD = 'TractConverter.py -i fibers_before_all.trk -o fibers_before_all.tck -a fsltensor_ftrial_FA.nii.gz'
subprocess.call(CMD,shell = True)

In [ ]:
CMD='tckmap fibers_before_all.tck syn_CLARITY_before.nii.gz -vox 0.015'
subprocess.call(CMD, shell = True)

## function for comparison of two tractography on a voxel base

In [ ]:
def compClarity(volume1,volume2):
    error = 0.0
    for x in range(volume1.shape[0]):
        for y in range(volume1.shape[1]):
            for z in range(volume1.shape[2]):
                error = error + np.abs(volume1[x,y,z]-volume2[x,y,z])
    return error

## apply COMMIT to the generated tractography

In [ ]:
path_commit = '/home/zhiwei/Desktop/semester-projects/results_script1/COMMIT'
path = '/home/zhiwei/Desktop/semester-projects/results_script1'

In [ ]:
trk_file_before = '/home/zhiwei/Desktop/semester-projects/results_script1/fibers_before_all.trk'

In [ ]:
from commit import trk2dictionary
trk2dictionary.run(filename_trk   = trk_file_before ,path_out= path_commit )
import commit
commit.core.setup()

In [ ]:
mit=commit.Evaluation('.','results_script1')

In [ ]:
mit.load_data('/home/zhiwei/Desktop/semester-projects/results_script1/syn_CLARITY.nii.gz','/home/zhiwei/Desktop/semester-projects/results_script1/BVECTOR.scheme')
mit.set_model('VolumeFractions')
hasISO=False
mit.model.set(hasISO)
mit.generate_kernels(regenerate=True)

In [ ]:
mit.load_kernels()
mit.load_dictionary(path_commit)
mit.set_threads(1)

In [ ]:
mit.build_operator()

In [ ]:
mit.fit(max_iter = 100000000)
## mit.fit(tol_fun = 1e-7, max_iter = 100000000)

In [ ]:
mit.save_results(save_coeff=True)

In [ ]:
trk_file = os.path.join(path_commit,'dictionary_TRK_fibers.trk')
fib_tmp,trk_hdr_tmp=nib.trackvis.read(trk_file)

## generate tractography after applying COMMIT with weight > 0, and get fib_after,hdr_after

In [ ]:
f = open('/home/zhiwei/Desktop/semester-projects/results_script1/COMMIT/Results_VolumeFractions/xic.txt')


In [ ]:
import heapq
data = f.read().splitlines()
data = map(eval,data)
max_index_list = map(data.index, heapq.nlargest(500,data))

#j=0
#for items in range(len(fib_tmp)-1,-1,-1):
#    if data[j] == 0:
#        del fib_tmp[items]
#    j=j+1
#nib.trackvis.write('fibers_after.trk',fib_tmp,trk_hdr_tmp)

In [ ]:
j=0
for items in range(len(fib_tmp)-1,-1,-1):
    if items not in max_index_list:
        del fib_tmp[items]
nib.trackvis.write('fibers_after.trk',fib_tmp,trk_hdr_tmp)

In [ ]:
fib_after,hdr_after=nib.trackvis.read('/home/zhiwei/Desktop/semester-projects/results_script1/fibers_after.trk')

In [ ]:
CMD = 'TractConverter.py -i fibers_after.trk -o fibers_after.tck -a fsltensor_ftrial_FA.nii.gz'
subprocess.call(CMD,shell = True)

In [ ]:
CMD='tckmap fibers_after.tck syn_CLARITY_after.nii.gz -vox 0.015'
subprocess.call(CMD, shell = True)

## downsample fibers(before/after) to the original one, take mean distance and std as measurement

## downsample the fibers and get tracks_before, tracks_after, tracks_original

In [ ]:
from dipy.tracking.metrics import downsample
import random
tracks_after = [downsample(fib_after[i][0],7) for i in range(len(fib_after))]
tracks_original = [downsample(fib_original[i][0],7) for i in range(len(fib_original))]
## generate a random list to get equal number of fibers from fib before COMMIT
list = range(len(fib_before))
slice1 = random.sample(list, len(fib_after))
tracks_before1 = [downsample(fib_before[i][0],7) for i in slice1]

In [ ]:
from dipy.segment.bundles import bundles_distances_mdf
DM_before1 = bundles_distances_mdf(tracks_before1, tracks_original)
minD_before1 = [min(DM_before1[i,:]) for i in range(len(DM_before1))]
DM_after = bundles_distances_mdf(tracks_after, tracks_original)
minD_after = [min(DM_after[i,:]) for i in range(len(DM_after))]

## compare tracks_after and tracks_before with tracks_original with mean distance and STD

In [ ]:
meanD_before= np.mean(minD_before1)
stdD_before= np.std(minD_before1)

meanD_after= np.mean(minD_after)
stdD_after= np.std(minD_after)

In [ ]:
meanD_after

In [ ]:
meanD_before

## Compare on a voxel level

original resolution -- 0.015mm

for items in range(len(fib_before)-1,-1,-1):
    if items not in slice1:
        del fib_before[items]
    j=j+1
nib.trackvis.write('fibers_before.trk',fib_before,hdr_before)

fib_before,hdr_before=nib.trackvis.read('/home/zhiwei/Desktop/semester-projects/results_script1/fibers_before.trk')

CMD = 'TractConverter.py -i fibers_before.trk -o fibers_before.tck -a fsltensor_ftrial_FA.nii.gz'
subprocess.call(CMD,shell = True)

CMD='tckmap fibers_before.tck syn_CLARITY_random.nii.gz -vox 0.015' 
subprocess.call(CMD, shell = True)

data_original = nib.load('syn_CLARITY.nii.gz')
data_before = nib.load('syn_CLARITY_random.nii.gz')
data_after = nib.load('syn_CLARITY_after.nii.gz')
original = data_original.get_fdata()
random = data_before.get_fdata()
after = data_after.get_fdata()

## normalization
sum_original = sum(sum(sum(data_original.get_fdata())))
sum_before = sum(sum(sum(data_before.get_fdata())))
sum_after = sum(sum(sum(data_after.get_fdata())))
volume_original = data_original.get_fdata()/sum_original
volume_before = data_before.get_fdata()/sum_before
volume_after = data_after.get_fdata()/sum_after

## correlation measure to conduct grey level matching

center_original = (original-sum_original/(191*178*39)).flatten()
center_before = (random - sum_before/(191*178*39)).flatten()
center_after = (after - sum_after/(191*178*39)).flatten()

module_original = np.sqrt(np.dot(center_original,center_original))
module_before = np.sqrt(np.dot(center_before,center_before))
module_after = np.sqrt(np.dot(center_after,center_after))

random_coef = np.dot(center_original.flatten(),center_before.flatten())/(module_original*module_before)
COMMIT_coef = np.dot(center_original.flatten(),center_after.flatten())/(module_original*module_after)

print(random_coef)
print(COMMIT_coef)

square_error_before = compClarity(volume_before, volume_original)
square_error_after = compClarity(volume_after, volume_original)

print(square_error_before)
print(square_error_after)

## change the resolution here

resolution based on mean distance after COMMIT

CMD='tckmap origin0.015.tck syn_CLARITY_COMMIT_res.nii.gz -vox 0.195' 
subprocess.call(CMD, shell = True)

CMD='tckmap fibers_after.tck syn_CLARITY_after_COMMIT_res.nii.gz -vox 0.195'
subprocess.call(CMD, shell = True)

CMD='tckmap fibers_before.tck syn_CLARITY_random_COMMIT_res.nii.gz -vox 0.195'
subprocess.call(CMD, shell = True)

data_original_COMMIT_res = nib.load('syn_CLARITY_COMMIT_res.nii.gz')
data_before_COMMIT_res = nib.load('syn_CLARITY_random_COMMIT_res.nii.gz')
data_after_COMMIT_res = nib.load('syn_CLARITY_after_COMMIT_res.nii.gz')
## normalization
sum_original_COMMIT_res = sum(sum(sum(data_original_COMMIT_res.get_fdata())))
sum_before_COMMIT_res = sum(sum(sum(data_before_COMMIT_res.get_fdata())))
sum_after_COMMIT_res = sum(sum(sum(data_after_COMMIT_res.get_fdata())))
volume_original_COMMIT_res = data_original_COMMIT_res.get_fdata()/sum_original_COMMIT_res
volume_before_COMMIT_res = data_before_COMMIT_res.get_fdata()/sum_before_COMMIT_res
volume_after_COMMIT_res = data_after_COMMIT_res.get_fdata()/sum_after_COMMIT_res

square_error_before_COMMIT_res = compClarity(volume_before_COMMIT_res, volume_original_COMMIT_res)
square_error_after_COMMIT_res = compClarity(volume_after_COMMIT_res, volume_original_COMMIT_res)

## Compare on a voxel level - binary

original resolution -- 0.015mm

def binarize(volume):
    volume_prime = np.zeros([len(volume),len(volume[1]),len(volume[1][1])])
    for x in range(len(volume)):
        for y in range(len(volume[1])):
            for z in range(len(volume[1][1])):
                if volume[x,y,z]!=0:
                    volume_prime[x,y,z]=1
    return volume_prime

Volume_Original = binarize(volume_original)
Volume_Before = binarize(volume_before)
Volume_After = binarize(volume_after)
error_before = compClarity(Volume_Before, Volume_Original)
error_after = compClarity(Volume_After, Volume_Original)

print(error_before)
print(error_after)


## change the resolution here

COMMIT resolution 0.186

Volume_Original_COMMIT_res = binarize(volume_original_COMMIT_res)
Volume_Before_COMMIT_res = binarize(volume_before_COMMIT_res)
Volume_After_COMMIT_res = binarize(volume_after_COMMIT_res)
error_before_COMMIT_res = compClarity(Volume_Before_COMMIT_res, Volume_Original_COMMIT_res)
error_after_COMMIT_res = compClarity(Volume_After_COMMIT_res, Volume_Original_COMMIT_res)

## save results

In [ ]:
cd ..

In [ ]:
f = open('meanD_RANDOM_COMMIT.txt','a')  
f.write(str(meanD_before)+', ')
f.write(str(meanD_after)+', ')
f.close()

f = open('stdD_RANDOM_COMMIT.txt','a') 
f.write(str(stdD_before)+', ')
f.write(str(stdD_after)+', ')
f.close()

f = open('VOXEL_B_random_COMMIT.txt','a')
f.write(str(error_before)+', ')
f.write(str(error_after)+', ')
f.close()

f = open('VOXEL_NB_random_commit.txt','a')
f.write(str(square_error_before)+', ')
f.write(str(square_error_after)+', ')
f.close()


f = open('coef_random_COMMIT.txt','a')
f.write(str(random_coef)+', ')
f.write(str(COMMIT_coef)+', ')
f.close()

f = open('meanD_std3000.txt','a')
f.write('\n')
f.write('compare tractography on a voxel level_COMMIT_res - BINARIZATION:')
f.write('\nerror random: ' + str(error_before_COMMIT_res))
f.write('\nerror COMMIT: ' + str(error_after_COMMIT_res)) 
f.write('\n\n')
f.close()

f = open('meanD_std3000.txt','a')
f.write('\n')
f.write('compare tractography on a voxel level_COMMIT_res- non-BINARIZATION:')
f.write('\nsquare error random: ' + str(square_error_before_COMMIT_res))
f.write('\nsquare error COMMIT: ' + str(square_error_after_COMMIT_res)) 
f.write('\n\n\n\n\n')
f.close()

In [ ]:
os.rename('results_script1','5_PERCENTER_%d'%order)